In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(temperature=0.1) 

prompt = "Cost of $355.43 + $924.85 + $5000.9 + $1940.9 + $550.33 + $440.44 + $333.33 + $22222.22"

llm.invoke(prompt)

/Users/jeongmin/PycharmProjects/langchain-practice/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


AIMessage(content='= $355.43 + $924.85 + $5000.9 + $1940.9 + $550.33 + $440.44 + $333.33 + $22222.22\n= $30,768.00', response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 51, 'total_tokens': 102}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-89566056-8ba1-41bb-9c24-86130a8ffc9d-0')

In [7]:
def plus(a, b): 
    return a + b 

agent = initialize_agent(llm=llm, verbose=True, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, tools=[
    StructuredTool.from_function(
        func=plus, 
        name="Sum Calculator",
        description="Use this to perform sums of two numbers. this tool take two arguments, both should be numbers"
    )
])

agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought: I can use the Sum Calculator tool to find the total cost of all these amounts.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 355.43, "b": 924.85}
}
```
Observation: 1280.28
Thought:I have calculated the sum of the first two amounts correctly. Now, I will continue to calculate the total cost by adding the remaining amounts one by one using the Sum Calculator tool. Let's proceed with the next calculation. 
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 1280.28, "b": 5000.9}
}
```
Observation: 6281.179999999999
Thought:I have calculated the sum of the first three amounts correctly. Now, I will continue to calculate the total cost by adding the remaining amounts one by one using the Sum Calculator tool. Let's proceed with the next calculation. 
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 6281.18, "b": 1940.9}
}
```
Observation: 8222.08
Thought:I have calculated the sum of the

{'input': 'Cost of $355.43 + $924.85 + $5000.9 + $1940.9 + $550.33 + $440.44 + $333.33 + $22222.22',
 'output': 'The total cost of $355.43 + $924.85 + $5000.9 + $1940.9 + $550.33 + $440.44 + $333.33 + $22222.22 is $31768.4.'}

In [16]:
from typing import Any, Type
from langchain.tools import BaseTool
from pydantic import BaseModel, Field

class CalculatorToolArgsSchema(BaseModel): 
    a: int = Field(description="The first number")
    b: int = Field(description="The second number") 
    
class CalculatorTool(BaseTool): 
    name = "CalculatorTool"
    description = """
        Use this to perform sums of two numbers. 
        The first and second arguments should be numbers.  
        Only receives two arguments
    """
    args_schema : Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return plus(a, b) 
    

prompt = "Cost of $35543 + $92485 + $50009 + $1940.9 + $55033 + $44044 + $33333 + $2222222"
    
agent = initialize_agent(llm=llm, verbose=True, agent=AgentType.OPENAI_FUNCTIONS, tools=[CalculatorTool()])

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 35543, 'b': 92485}`


128028
Invoking: `CalculatorTool` with `{'a': 128028, 'b': 50009}`


178037
Invoking: `CalculatorTool` with `{'a': 178037, 'b': 1940.9}`




ValidationError: 1 validation error for CalculatorToolArgsSchema
b
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=1940.9, input_type=float]
    For further information visit https://errors.pydantic.dev/2.7/v/int_from_float

In [9]:
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.prompts import PromptTemplate
from typing import Any, Type
from langchain.tools import BaseTool
from pydantic import BaseModel, Field

class StockMarketSymbolSearchToolArgsSchema(BaseModel): 
    query: str = Field(description="The query you will search for ")

class StockMarketSymbolSearchTool(BaseTool): 
    name = "StockMarketSymbolSearchTool"
    description = """
        Use this tool to find the stock market symbol for a company. 
        It takes a query as an argument. 
        Example query: Stock Market Symbol for Apple Company. 
    """
    
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema
    
    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)
    
template = """
    Give me information on {company_name} stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."
"""

prompt = PromptTemplate.from_template(template)

prompt = prompt.format(company_name="Cloudflare")

agent = initialize_agent(llm=llm, verbose=True, agent=AgentType.OPENAI_FUNCTIONS, tools=[StockMarketSymbolSearchTool()])

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock'}`


Cloudflare's stock has recently been incredibly volatile, ranging between approximately $30 and $80. The significant price movements have been in response to news and events affecting the broader tech industry. Cloudflare operates in the highly competitive cybersecurity industry, characterized by rapid technological change, intense competition ... Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. Internet security and content delivery network Cloudflare (NYSE:NET) beat analysts' expectations in Q4 FY2023, with revenue up 32% year on year to $362.5 million. The company expects next quarter ... Indeed, Cloudflare stock has rallied nearly 80% over the past year, fueled by demand for its cybersecurity solutions, its content delivery network platform, and increased web traffi

{'input': '\n    Give me information on Cloudflare stock and help me analyze if it\'s a potential good investment."\n',
 'output': "Cloudflare Inc. (NYSE: NET) is a company operating in the cybersecurity industry, providing internet security and content delivery network services. In the most recent quarter, Cloudflare beat analysts' expectations with revenue up 32% year on year to $362.5 million. The stock has rallied nearly 80% over the past year, driven by demand for its cybersecurity solutions, content delivery network platform, and increased web traffic.\n\nWhen analyzing Cloudflare as a potential investment, it's important to consider the following factors:\n1. **Industry Outlook**: The cybersecurity industry is expected to continue growing as businesses and individuals increasingly rely on digital platforms.\n2. **Financial Performance**: Cloudflare's revenue growth and profitability are key indicators of its financial health.\n3. **Competitive Position**: Evaluate Cloudflare's c

In [17]:
import os
import requests

load_dotenv()

llm = ChatOpenAI(temperature=0.1, model_name="gpt-4o-2024-05-13") 

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(description="Stock symbol of the company. Example: AAPL, TSLA")

class CompanyOverviewTool(BaseTool): 
    name = "CompanyOverview"
    description = """    
    Use this to get an overview of the financials of the company. 
    You should enter a stock symbol. 
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    
    def _run(self,symbol): 
        r = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()

class CompanyIncomeStatementTool(BaseTool): 
    name = "CompanyIncomeStatement"
    description = """    
    Use this to get the income statement of the financials of the company. 
    You should enter a stock symbol. 
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    
    def _run(self,symbol): 
        r = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()["annualReports"]

class CompanyStockPerformanceTool(BaseTool): 
    name = "CompanyStockPerformance"
    description = """    
    Use this to get the weekly performance of a company stock.  
    You should enter a stock symbol. 
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    
    def _run(self,symbol): 
        r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()

    
template = """
    Give me information on {company_name} stock, considering its financials and income statements and stock performance and help me analyze if it's a potential good investment."
"""

prompt = PromptTemplate.from_template(template)

prompt = prompt.format(company_name="Cloudflare")

agent = initialize_agent(llm=llm, verbose=True, agent=AgentType.OPENAI_FUNCTIONS, tools=[StockMarketSymbolSearchTool(), CompanyOverviewTool(), CompanyIncomeStatementTool(), CompanyStockPerformanceTool()])

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare'}`


Read the latest posts from Cloudflare's official blog, covering topics such as security, performance, AI, DNS, email, and more. Learn about Cloudflare's new features, acquisitions, partnerships, and events. Cloudflare is one of the world's largest connectivity cloud networks . Today, anyone with an Internet presence can have faster and more secure websites and applications thanks to Cloudflare. This includes bloggers, businesses, and even non-profits. Millions of Internet properties are on Cloudflare, and our network is growing by tens of ... Cloudflare's Year in Review revealed that in 2023 alone, there was a 25% increase in global traffic, with no signs of slowing. This growth underscores the dependency on Internet services to facilitate and underpin vital systems and tasks such as supporting global digital economies, enabling the operations of healthcare networks, mainta

{'input': '\n    Give me information on Cloudflare stock, considering its financials and income statements and stock performance and help me analyze if it\'s a potential good investment."\n',
 'output': "### Cloudflare Inc. (NET) Overview\n\n**Company Description:**\nCloudflare Inc. operates a cloud platform that offers a range of network services to companies around the world. The company is headquartered in San Francisco, California.\n\n**Key Financial Metrics:**\n- **Market Capitalization:** $25.10 Billion\n- **EBITDA:** -$79.94 Million\n- **P/E Ratio:** None (due to negative earnings)\n- **PEG Ratio:** 2.355\n- **Book Value:** $2.347\n- **EPS (TTM):** -$0.52\n- **Revenue (TTM):** $1.39 Billion\n- **Gross Profit (TTM):** $742.63 Million\n- **Profit Margin:** -13.1%\n- **Operating Margin:** -14.4%\n- **Return on Assets (TTM):** -4.41%\n- **Return on Equity (TTM):** -25%\n- **Quarterly Revenue Growth (YOY):** 30.5%\n- **Analyst Target Price:** $90.26\n- **Analyst Ratings:** \n  - Stro

In [21]:
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(temperature=0.1, model_name="gpt-4o-2024-05-13") 
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm, 
    toolkit=toolkit, 
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.invoke("Give me the 5 directors that ")
